In [494]:
#!pip install yfinance
#!pip install pytickersymbols

In [495]:
import yfinance
import pandas as pd
import numpy as np
import math
from scipy import stats
import matplotlib.pyplot as plt
from pytickersymbols import PyTickerSymbols
from datetime import datetime, timedelta

In [496]:
#APLICANDO ETF'S
stocks_tech = ['QBAK', 'AVT']

#['JMP', 'OPY'], ['PEP', 'KO'], ['PCG', 'WEC'], ['PCG', 'UTL'], ['ED', 'DUK'], ['PCG', 'XEL'], ['WEC', 'XEL'], ['NTAP', 'RADA']
#['TAIT', 'IIVI']***, ['TAIT', 'ITRN']***, ['QBAK', 'ITRN'], ['QBAK', 'AVT']

In [497]:
mv_avg = 10
factor = .1 #Factor +- para los spreads
cash=2500 #por Trade
#period

In [498]:
d = pd.DataFrame([])
d_open = pd.DataFrame([])
for j in range(len(stocks_tech)):

  d.insert(j,stocks_tech[j], yfinance.download(stocks_tech[j],period='10y')['Close'])
  d_open.insert(j,stocks_tech[j], yfinance.download(stocks_tech[j],period='10y')['Open'])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [499]:
d_mv_avg = d.rolling(mv_avg).mean().dropna()#[-5:]
d_mv_avg.head(3)

,QBAK,AVT
Date,,
2011-02-28,10.800,35.616
2011-03-01,10.818,35.288
2011-03-02,10.824,34.967


In [500]:
d_open.rolling(mv_avg).mean().dropna()#[-5:]
d_open = d_open.loc[d_mv_avg.index[0].strftime('%Y-%m-%d'):d_mv_avg.index[-1].strftime('%Y-%m-%d')]
d_open.head(3)

,QBAK,AVT
Date,,
2011-02-28,11.10,34.700001
2011-03-01,11.10,34.380001
2011-03-02,11.04,33.400002


In [501]:
d_actuales = d.loc[d_mv_avg.index[0].strftime('%Y-%m-%d'):d_mv_avg.index[-1].strftime('%Y-%m-%d')]#[-5:]
d_actuales.head(3)

,QBAK,AVT
Date,,
2011-02-28,11.04,34.209999
2011-03-01,11.04,33.340000
2011-03-02,10.80,33.590000


In [502]:
# print((d_mv_avg.iloc[-1,1]-d_mv_avg.iloc[-1,4])*(1+factor))
# print(d_mv_avg.iloc[-1,1]-d_mv_avg.iloc[-1,4])
# print((d_mv_avg.iloc[-1,1]-d_mv_avg.iloc[-1,4])*(1-factor))

In [503]:
#d_actuales.iloc[-1,1]-d_actuales.iloc[-1,4]

In [504]:
titulos = d.columns.values[0:].tolist()

In [505]:
%%time

iteraciones = 0
greens = 0
reds = 0
PL = 0
    
for h in range(1,len(d_actuales)):
  posiciones = []
  for j in range(0,len(d_actuales.iloc[h-1])):
    
    for i in range(0,len(titulos)):
      #Condición de que el par no sean el mismo símbolo, y de que no se repita el par (e.g. AAPL - CSCO y CSCO - AAPL)
      if titulos[j] != titulos[i] and i+j <= len(titulos): #and titulos[i] not in posiciones:
        spread_curr = d_actuales.iloc[h-1][j]-d_actuales.iloc[h-1][i]
        spread_avg = d_mv_avg.iloc[h-1][j]-d_mv_avg.iloc[h-1][i]

        spread_alto = spread_avg * (1+factor)
        spread_bajo = spread_avg * (1-factor)

        iteraciones = iteraciones + 1
        #CONDICION EN LA QUE EL SPREAD DE LOS MOVING AVERAGES ES POSITIVO
        if spread_avg > 0:
          num_shares_j = cash//d_open.iloc[h,j]
          num_shares_i = cash//d_open.iloc[h,i]
          if spread_curr > spread_alto:
            compra = num_shares_i*(d_actuales.iloc[h,i]-d_open.iloc[h,i])
            short = num_shares_j*(d_open.iloc[h,j]-d_actuales.iloc[h,j])
            if compra >= 0 and titulos[i] not in posiciones: 
              greens = greens + 1
              PL = PL + compra
              posiciones.append(titulos[i])
            elif titulos[i] not in posiciones: 
              reds = reds + 1 
              PL = PL + compra
              posiciones.append(titulos[i])
            if short >= 0 and titulos[j] not in posiciones: 
              greens = greens +1
              PL = PL + short
              posiciones.append(titulos[j])
            elif titulos[j] not in posiciones: 
              reds = reds + 1
              PL = PL + short
              posiciones.append(titulos[j])
            print(d_mv_avg.index[h-1].strftime('%Y-%m-%d'), 'Spread Positivo de:',  spread_curr, '; Short:', titulos[j], 'y Compra:', titulos[i])
            print('RESULTADOS: Short', short, 'Compra', compra)
          elif spread_curr < spread_bajo:
            compra = num_shares_j*(d_actuales.iloc[h,j]-d_open.iloc[h,j])
            short = num_shares_i*(d_open.iloc[h,i]-d_actuales.iloc[h,i])
            if compra >= 0 and titulos[j] not in posiciones: 
              greens = greens + 1
              PL = PL + compra
              posiciones.append(titulos[j])
            elif titulos[j] not in posiciones: 
              reds = reds + 1 
              PL = PL + compra
              posiciones.append(titulos[j])
            if short >= 0 and titulos[i] not in posiciones: 
              greens = greens +1
              PL = PL + short
              posiciones.append(titulos[i])
            elif titulos[i] not in posiciones: 
              reds = reds + 1
              PL = PL + short
              posiciones.append(titulos[i])
            print(d_mv_avg.index[h-1].strftime('%Y-%m-%d'), 'Spread Positivo de:',  spread_curr, '; Compra:', titulos[j], 'y Short:', titulos[i])
            print('RESULTADOS: Compra', compra, 'Short', short)

        #CONDICION EN LA QUE EL SPREAD DE LOS MOVING AVERAGES ES NEGATIVO (CAMBIAN LOS VALORES CON FACTORES)
        elif spread_avg < 0:
          num_shares_j = cash//d_open.iloc[h,j]
          num_shares_i = cash//d_open.iloc[h,i]
          if spread_curr < spread_alto:
            compra = num_shares_j*(d_actuales.iloc[h,j]-d_open.iloc[h,j])
            short = num_shares_i*(d_open.iloc[h,i]-d_actuales.iloc[h,i])
            if compra >= 0 and titulos[j] not in posiciones: 
              greens = greens + 1
              PL = PL + compra
              posiciones.append(titulos[j])
            elif titulos[j] not in posiciones: 
              reds = reds + 1 
              PL = PL + compra
              posiciones.append(titulos[j])
            if short >= 0 and titulos[i] not in posiciones: 
              greens = greens +1
              PL = PL + short
              posiciones.append(titulos[i])
            elif titulos[i] not in posiciones: 
              reds = reds + 1
              PL = PL + short
              posiciones.append(titulos[i])
            print(d_mv_avg.index[h-1].strftime('%Y-%m-%d'), 'Spread Negativo de:',  spread_curr, '; Compra:', titulos[j], 'y Short:', titulos[i])
            print('RESULTADOS: Compra', compra, 'Short', short)
          elif spread_curr > spread_bajo:
            compra = num_shares_i*(d_actuales.iloc[h,i]-d_open.iloc[h,i])
            short = num_shares_j*(d_open.iloc[h,j]-d_actuales.iloc[h,j])
            if compra >= 0 and titulos[i] not in posiciones: 
              greens = greens + 1
              PL = PL + compra
              posiciones.append(titulos[i])
            elif titulos[i] not in posiciones: 
              reds = reds + 1 
              PL = PL + compra
              posiciones.append(titulos[i])
            if short >= 0 and titulos[j] not in posiciones: 
              greens = greens +1
              PL = PL + short
              posiciones.append(titulos[j])
            elif titulos[j] not in posiciones: 
              reds = reds + 1
              PL = PL + short
              posiciones.append(titulos[j])
            print(d_mv_avg.index[h-1].strftime('%Y-%m-%d'), 'Spread Negativo de:',  spread_curr, '; Short:', titulos[j], 'y Compra:', titulos[i])
            print('RESULTADOS: Short', short, 'Compra', compra)    

2011-06-06 Spread Negativo de: -21.390000343322754 ; Short: QBAK y Compra: AVT
RESULTADOS: Short -56.15994644165039 Compra 18.479835510253906
2011-06-06 Spread Positivo de: 21.390000343322754 ; Compra: AVT y Short: QBAK
RESULTADOS: Compra 18.479835510253906 Short -56.15994644165039
2011-06-08 Spread Negativo de: -20.690000534057617 ; Short: QBAK y Compra: AVT
RESULTADOS: Short -110.8798942565918 Compra 9.360065460205078
2011-06-08 Spread Positivo de: 20.690000534057617 ; Compra: AVT y Short: QBAK
RESULTADOS: Compra 9.360065460205078 Short -110.8798942565918
2011-06-09 Spread Negativo de: -20.49000072479248 ; Short: QBAK y Compra: AVT
RESULTADOS: Short -26.399974822998047 Compra -17.939964294433594
2011-06-09 Spread Positivo de: 20.49000072479248 ; Compra: AVT y Short: QBAK
RESULTADOS: Compra -17.939964294433594 Short -26.399974822998047
2011-06-10 Spread Negativo de: -19.97000026702881 ; Short: QBAK y Compra: AVT
RESULTADOS: Short 91.56001663208008 Compra -42.660072326660156
2011-06-10

In [506]:
iteraciones

5014

In [507]:
print(greens)
print(reds)
print(PL)

71
53
1912.5395426750183
